## Setting up the development environment

In [1]:
!pip install cohere tiktoken openai
!pip install langchain

     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 48.2/48.2 kB 2.0 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 2.0/2.0 MB 15.4 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 77.0/77.0 kB 7.9 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 2.7/2.7 MB 30.7 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 1.9/1.9 MB 13.1 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 41.9/41.9 kB 4.6 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 49.4/49.4 kB 5.3 MB/s eta 0:00:00


## Loading data

In [2]:
!pip install python-dotenv

In [3]:
import openai
import os
from dotenv import load_dotenv, find_dotenv

_ = load_dotenv(find_dotenv())
openai.api_key  = os.environ["OPENAI_API_KEY"]

In [4]:
import pandas as pd
pd.set_option('display.max_colwidth', None)

file_name = "conference_session_info.csv"

df = pd.read_csv(file_name)
df.shape

(30, 8)

In [5]:
df.head()

,Start Date,End Date,Session Name,Session Description,Session Track,Industry,Speaker Name,Room Name
0,07/27/2020 02:00 PM,07/27/2020 03:30 PM,3D Printing for the Non-Tech Minded,This is 3D Printing 101 for those in the makerspace that don’t consider themselves technically astute.,3D Printing and Design,Technology,Jeffery Lowe & Marysa Balma,Room 101
1,07/27/2020 02:00 PM,07/27/2020 03:00 PM,3D Printing with Clay,"Clay has historically been a hands-on medium for over 20,000 years, both to create practical items for day-to-day living, and art for day-to-day beauty. Now with the advent of commercially available 3D clay printers, artists and engineers alike are creating inspirational pieces that were previously unimaginable.",3D Printing and Design,Education,Julie Parker,Room 201
2,07/27/2020 02:00 PM,07/27/2020 03:30 PM,Art in the Age of Automation,"There are some people who don’t believe that art can be “art” if it is made by a machine. The most intriguing and sometimes surprising beautiful art is made by non-sentient robots, based on data and interpretations of that data. So what are artists afraid of?",Ethics and Environment,Technology,Jamill Waters & Jess Abbott,Room 103
3,07/27/2020 02:00 PM,07/27/2020 03:30 PM,Augmented Real(ity) Estate,"Imagine if your company is moving you to a state too far away to spend time looking for a new place to live. Wouldn't it be nice to be able to meet an agent, walk through a house, open doors, go up steps, and check out the neighborhood from the comfort of your couch? Check out the latest innovations in augmented reality in the real estate market, and discuss its economic benefits.",Virtual and Augmented Reality,Technology,Grant Jacobson,Room 104
4,07/27/2020 02:00 PM,07/27/2020 03:00 PM,Hands-On Hacks,Join your fellow makers in learning their favorite hacks in popular maker categories.,Education and Training,Education,David Powlowski,Grand View Hall


### Load a CSV file into a list of Documents

Each document represents one row of the CSV file. Every row is converted into a key/value pair and outputted to a new line in the document’s page_content.

Reference: https://api.python.langchain.com/en/latest/document_loaders/langchain.document_loaders.csv_loader.CSVLoader.html

In [6]:
from langchain.document_loaders.csv_loader import CSVLoader

loader = CSVLoader(file_path=file_name)
docs = loader.load()

In [7]:
len(docs)

30

In [8]:
print(docs[0].page_content[:500])

Start Date: 07/27/2020 02:00 PM
End Date: 07/27/2020 03:30 PM
Session Name: 3D Printing for the Non-Tech Minded
Session Description: This is 3D Printing 101 for those in the makerspace that don’t consider themselves technically astute.
Session Track: 3D Printing and Design
Industry: Technology
Speaker Name: Jeffery Lowe & Marysa Balma
Room Name: Room 101


## Split documents

In [9]:
from langchain.text_splitter import RecursiveCharacterTextSplitter, CharacterTextSplitter

chunk_size = 512
chunk_overlap = 32

c_text_splitter = CharacterTextSplitter(
    separator="\n",
    chunk_size=chunk_size,
    chunk_overlap=chunk_overlap,
    length_function=len
)

r_text_splitter = RecursiveCharacterTextSplitter(
    chunk_size = chunk_size,
    chunk_overlap  = chunk_overlap,
    length_function = len,
    add_start_index = True,
)

In [10]:
pages = c_text_splitter.split_documents(docs)

print(pages[0])
print(pages[1])

page_content='Start Date: 07/27/2020 02:00 PM\nEnd Date: 07/27/2020 03:30 PM\nSession Name: 3D Printing for the Non-Tech Minded\nSession Description: This is 3D Printing 101 for those in the makerspace that don’t consider themselves technically astute.\nSession Track: 3D Printing and Design\nIndustry: Technology\nSpeaker Name: Jeffery Lowe & Marysa Balma\nRoom Name: Room 101' metadata={'source': 'conference_session_info.csv', 'row': 0}
page_content='Start Date: 07/27/2020 02:00 PM\nEnd Date: 07/27/2020 03:00 PM\nSession Name: 3D Printing with Clay\nSession Description: Clay has historically been a hands-on medium for over 20,000 years, both to create practical items for day-to-day living, and art for day-to-day beauty. Now with the advent of commercially available 3D clay printers, artists and engineers alike are creating inspirational pieces that were previously unimaginable.\nSession Track: 3D Printing and Design\nIndustry: Education' metadata={'source': 'conference_session_info.csv'

In [11]:
pages = r_text_splitter.split_documents(docs)

print(pages[0])
print(pages[1])

page_content='Start Date: 07/27/2020 02:00 PM\nEnd Date: 07/27/2020 03:30 PM\nSession Name: 3D Printing for the Non-Tech Minded\nSession Description: This is 3D Printing 101 for those in the makerspace that don’t consider themselves technically astute.\nSession Track: 3D Printing and Design\nIndustry: Technology\nSpeaker Name: Jeffery Lowe & Marysa Balma\nRoom Name: Room 101' metadata={'source': 'conference_session_info.csv', 'row': 0, 'start_index': 0}
page_content='Start Date: 07/27/2020 02:00 PM\nEnd Date: 07/27/2020 03:00 PM\nSession Name: 3D Printing with Clay\nSession Description: Clay has historically been a hands-on medium for over 20,000 years, both to create practical items for day-to-day living, and art for day-to-day beauty. Now with the advent of commercially available 3D clay printers, artists and engineers alike are creating inspirational pieces that were previously unimaginable.\nSession Track: 3D Printing and Design\nIndustry: Education' metadata={'source': 'conference

In [12]:
len(docs)

30

In [13]:
len(pages)

46

## Vectorstore and embedding

In [14]:
!pip install chromadb

     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 448.1/448.1 kB 6.6 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 2.4/2.4 MB 18.1 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 92.9/92.9 kB 11.5 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 59.5/59.5 kB 7.3 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 5.4/5.4 MB 45.7 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 6.2/6.2 MB 43.8 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 3.8/3.8 MB 19.8 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 67.3/67.3 kB 5.4 MB/s eta 0:00:00
  Installing build dependencies ... done
  Getting requirements to build wheel ... done
  Preparing metadata (pyproject.toml) ... done
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 593.7/593.7 kB 46.5 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 67.0/67.0 kB 7.9 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 46.0/46.0 kB 5.2 

In [15]:
from langchain.vectorstores import Chroma
from langchain.embeddings.openai import OpenAIEmbeddings

embedding = OpenAIEmbeddings()

In [16]:
persist_directory = 'persist_chroma'

In [17]:
vectordb = Chroma.from_documents(
    documents=pages,
    embedding=embedding,
    persist_directory=persist_directory
)

In [18]:
print(vectordb._collection.count())

46


In [19]:
len(pages)

46

In [20]:
vectordb.persist()

In [21]:
question = "which sessions are about augmented reality?"

In [22]:
docs = vectordb.similarity_search(question,k=3)
docs

[Document(page_content="Start Date: 07/27/2020 02:00 PM\nEnd Date: 07/27/2020 03:30 PM\nSession Name: Augmented Real(ity) Estate\nSession Description: Imagine if your company is moving you to a state too far away to spend time looking for a new place to live. Wouldn't it be nice to be able to meet an agent, walk through a house, open doors, go up steps, and check out the neighborhood from the comfort of your couch? Check out the latest innovations in augmented reality in the real estate market, and discuss its economic benefits.", metadata={'row': 3, 'source': 'conference_session_info.csv', 'start_index': 0}),
 Document(page_content='Session Track: Virtual and Augmented Reality\nIndustry: Technology\nSpeaker Name: Grant Jacobson\nRoom Name: Room 104', metadata={'row': 3, 'source': 'conference_session_info.csv', 'start_index': 508}),
 Document(page_content='Session Track: Virtual and Augmented Reality\nIndustry: Technology\nSpeaker Name: Teena Judkins\nRoom Name: Room 200', metadata={'r

## Retrieval with MMR

semantic search vs maximal marginal relevance

### Load a persist vectordb

In [23]:
vectordb = Chroma(
    persist_directory=persist_directory,
    embedding_function=embedding
)

In [24]:
docs_ss = vectordb.similarity_search(question,k=3)
docs_ss

[Document(page_content="Start Date: 07/27/2020 02:00 PM\nEnd Date: 07/27/2020 03:30 PM\nSession Name: Augmented Real(ity) Estate\nSession Description: Imagine if your company is moving you to a state too far away to spend time looking for a new place to live. Wouldn't it be nice to be able to meet an agent, walk through a house, open doors, go up steps, and check out the neighborhood from the comfort of your couch? Check out the latest innovations in augmented reality in the real estate market, and discuss its economic benefits.", metadata={'row': 3, 'source': 'conference_session_info.csv', 'start_index': 0}),
 Document(page_content='Session Track: Virtual and Augmented Reality\nIndustry: Technology\nSpeaker Name: Grant Jacobson\nRoom Name: Room 104', metadata={'row': 3, 'source': 'conference_session_info.csv', 'start_index': 508}),
 Document(page_content='Session Track: Virtual and Augmented Reality\nIndustry: Technology\nSpeaker Name: Teena Judkins\nRoom Name: Room 200', metadata={'r

In [25]:
docs_mmr = vectordb.max_marginal_relevance_search(question,k=3)
docs_mmr

[Document(page_content="Start Date: 07/27/2020 02:00 PM\nEnd Date: 07/27/2020 03:30 PM\nSession Name: Augmented Real(ity) Estate\nSession Description: Imagine if your company is moving you to a state too far away to spend time looking for a new place to live. Wouldn't it be nice to be able to meet an agent, walk through a house, open doors, go up steps, and check out the neighborhood from the comfort of your couch? Check out the latest innovations in augmented reality in the real estate market, and discuss its economic benefits.", metadata={'row': 3, 'source': 'conference_session_info.csv', 'start_index': 0}),
 Document(page_content='Session Track: Virtual and Augmented Reality\nIndustry: Technology\nSpeaker Name: Teena Judkins\nRoom Name: Room 200', metadata={'row': 8, 'source': 'conference_session_info.csv', 'start_index': 504}),
 Document(page_content='Industry: Technology\nSpeaker Name: Griffin Snow, Jarrod Anderson & Stephanie Watson\nRoom Name: Room 200', metadata={'row': 13, 'so

## Question answering


### Using default prompt

In [26]:
from langchain.chains import RetrievalQA
from langchain.chat_models import ChatOpenAI

import langchain
langchain.verbose = True

llm_name = "gpt-3.5-turbo"
llm = ChatOpenAI(model_name=llm_name, temperature=1)

In [27]:
qa_chain_default = RetrievalQA.from_chain_type(
    llm,
    retriever=vectordb.as_retriever(search_kwargs={"k": 3}),
    chain_type="stuff",
    return_source_documents=True
)

In [28]:
question = "Who talks about robot dogs in their session? and what is the session description?"

result = qa_chain_default({"query": question})
result



> Entering new RetrievalQA chain...


> Entering new StuffDocumentsChain chain...


> Entering new LLMChain chain...
Prompt after formatting:
System: Use the following pieces of context to answer the users question. 
If you don't know the answer, just say that you don't know, don't try to make up an answer.
----------------
Start Date: 07/27/2020 04:00 PM
End Date: 07/27/2020 05:00 PM
Session Name: Animals and AI
Session Description: We've known for years that robot dogs can never take the place of real dogs. But can robot people take the place of dog owners? We'll compare the plusses and minuses of the emerging field of Artificial Dogsitters.
Session Track: AI and Robotics
Industry: Technology
Speaker Name: Hui Bashirian
Room Name: Room 100

Start Date: 07/27/2020 02:00 PM
End Date: 07/27/2020 03:30 PM
Session Name: Art in the Age of Automation
Session Description: There are some people who don’t believe that art can be “art” if it is made by a machine. The most intriguing and somet

{'query': 'Who talks about robot dogs in their session? and what is the session description?',
 'result': 'Hui Bashirian talks about robot dogs in their session titled "Animals and AI." The session description is: "We\'ve known for years that robot dogs can never take the place of real dogs. But can robot people take the place of dog owners? We\'ll compare the plusses and minuses of the emerging field of Artificial Dogsitters."',
 'source_documents': [Document(page_content="Start Date: 07/27/2020 04:00 PM\nEnd Date: 07/27/2020 05:00 PM\nSession Name: Animals and AI\nSession Description: We've known for years that robot dogs can never take the place of real dogs. But can robot people take the place of dog owners? We'll compare the plusses and minuses of the emerging field of Artificial Dogsitters.\nSession Track: AI and Robotics\nIndustry: Technology\nSpeaker Name: Hui Bashirian\nRoom Name: Room 100", metadata={'row': 11, 'source': 'conference_session_info.csv', 'start_index': 0}),
  Do

In [29]:
print(result.get("result"))

Hui Bashirian talks about robot dogs in their session titled "Animals and AI." The session description is: "We've known for years that robot dogs can never take the place of real dogs. But can robot people take the place of dog owners? We'll compare the plusses and minuses of the emerging field of Artificial Dogsitters."


In [30]:
def pretty_print(text, words_per_line=15):
    # Split the input text into words
    words = text.split()

    # Iterate through the words
    for i in range(0, len(words), words_per_line):
        # Join the words
        line = ' '.join(words[i:i+words_per_line])
        print(line)

In [31]:
pretty_print(result.get("result"))

Hui Bashirian talks about robot dogs in their session titled "Animals and AI." The session
description is: "We've known for years that robot dogs can never take the place of
real dogs. But can robot people take the place of dog owners? We'll compare the
plusses and minuses of the emerging field of Artificial Dogsitters."


### Custom prompt template

In [32]:
from langchain.prompts import PromptTemplate

prompt_template = """You are a helpful assistant use the following pieces of context
from a private dataset to answer user's question.
If you don't know the answer, reply "No direct answer found."
Be concise.

###
Context: {context}
Question: {question}
Answer:"""


prompt_template = """Use the following pieces of context to answer the users question.
If you don't know the answer, just say that you don't know, don't try to make up an answer.
###
{context}

Question: {question}
Answer:"""

QA_CHAIN_PROMPT = PromptTemplate.from_template(prompt_template)

In [33]:
qa_chain = RetrievalQA.from_chain_type(
    llm,
    retriever=vectordb.as_retriever(search_kwargs={"k": 3}),
    chain_type="stuff",
    return_source_documents=True,
    chain_type_kwargs={"prompt": QA_CHAIN_PROMPT},
)

In [34]:
result = qa_chain({"query": question})
result



> Entering new RetrievalQA chain...


> Entering new StuffDocumentsChain chain...


> Entering new LLMChain chain...
Prompt after formatting:
Use the following pieces of context to answer the users question.
If you don't know the answer, just say that you don't know, don't try to make up an answer.
###
Start Date: 07/27/2020 04:00 PM
End Date: 07/27/2020 05:00 PM
Session Name: Animals and AI
Session Description: We've known for years that robot dogs can never take the place of real dogs. But can robot people take the place of dog owners? We'll compare the plusses and minuses of the emerging field of Artificial Dogsitters.
Session Track: AI and Robotics
Industry: Technology
Speaker Name: Hui Bashirian
Room Name: Room 100

Start Date: 07/27/2020 02:00 PM
End Date: 07/27/2020 03:30 PM
Session Name: Art in the Age of Automation
Session Description: There are some people who don’t believe that art can be “art” if it is made by a machine. The most intriguing and sometimes surprising beauti

{'query': 'Who talks about robot dogs in their session? and what is the session description?',
 'result': 'Hui Bashirian talks about robot dogs in their session "Animals and AI". The session description is "We\'ve known for years that robot dogs can never take the place of real dogs. But can robot people take the place of dog owners? We\'ll compare the plusses and minuses of the emerging field of Artificial Dogsitters."',
 'source_documents': [Document(page_content="Start Date: 07/27/2020 04:00 PM\nEnd Date: 07/27/2020 05:00 PM\nSession Name: Animals and AI\nSession Description: We've known for years that robot dogs can never take the place of real dogs. But can robot people take the place of dog owners? We'll compare the plusses and minuses of the emerging field of Artificial Dogsitters.\nSession Track: AI and Robotics\nIndustry: Technology\nSpeaker Name: Hui Bashirian\nRoom Name: Room 100", metadata={'row': 11, 'source': 'conference_session_info.csv', 'start_index': 0}),
  Document(p

In [36]:
pretty_print(result.get("result"))

Hui Bashirian talks about robot dogs in their session "Animals and AI". The session description
is "We've known for years that robot dogs can never take the place of real
dogs. But can robot people take the place of dog owners? We'll compare the plusses
and minuses of the emerging field of Artificial Dogsitters."


### Custom system message and human message template

In [37]:
from langchain.schema import SystemMessage
from langchain.prompts import ChatPromptTemplate, HumanMessagePromptTemplate


system_msg = """You are a helpful assistant Lilis using the following
pieces of context from a private dataset to answer user's question.
If you don't know the answer, reply "No direct answer found."

Start your answer with a brief greeting, for example, "Hi I am chatbot Lilis. Thank you for asking."

Always finish your answer with "Let me know if you have any other questions."
Be concise."""

human_template = """Use the following pieces of context to answer the users question.
###
{context}

Question: {question}
Answer:"""

messages = ChatPromptTemplate.from_messages(
    [
         SystemMessage(content=system_msg),
         HumanMessagePromptTemplate.from_template(human_template)
    ]
)

In [38]:
qa_chain_custom = RetrievalQA.from_chain_type(
    llm,
    retriever=vectordb.as_retriever(search_kwargs={"k": 3}),
    chain_type="stuff",
    return_source_documents=True,
    chain_type_kwargs={"prompt": messages},
)

In [39]:
question = "which sessions talk about learning experience in the era of AI?"

In [40]:
result = qa_chain_custom({"query": question})
result



> Entering new RetrievalQA chain...


> Entering new StuffDocumentsChain chain...


> Entering new LLMChain chain...
Prompt after formatting:
System: You are a helpful assistant Lilis using the following
pieces of context from a private dataset to answer user's question.
If you don't know the answer, reply "No direct answer found."

Start your answer with a brief greeting, for example, "Hi I am chatbot Lilis. Thank you for asking."

Always finish your answer with "Let me know if you have any other questions."
Be concise.
Human: Use the following pieces of context to answer the users question.
###
Start Date: 07/27/2020 10:30 AM
End Date: 07/27/2020 12:00 PM
Session Name: AI and Education—Developing a Data Strategy
Session Description: According to research, the majority of higher education educators agree that AI will be a key part of their educational toolbox moving forward. The same research shows that nearly all of these educators have absolutely no clue how to build AI into their

{'query': 'which sessions talk about learning experience in the era of AI?',
 'result': 'Hi, I am chatbot Lilis. Thank you for asking.\n\nThe session that talks about learning experience in the era of AI is "LEX: Always Remember that the End Product is Human." It discusses how to develop new technology and experiences while keeping in mind that the end "product" is a human that is counting on you to learn.\n\nLet me know if you have any other questions.',
 'source_documents': [Document(page_content='Start Date: 07/27/2020 10:30 AM\nEnd Date: 07/27/2020 12:00 PM\nSession Name: AI and Education—Developing a Data Strategy\nSession Description: According to research, the majority of higher education educators agree that AI will be a key part of their educational toolbox moving forward. The same research shows that nearly all of these educators have absolutely no clue how to build AI into their programs. This session gives practical suggestions for getting started on an AI data strategy.', 

In [41]:
pretty_print(result.get("result"))

Hi, I am chatbot Lilis. Thank you for asking. The session that talks about learning
experience in the era of AI is "LEX: Always Remember that the End Product is
Human." It discusses how to develop new technology and experiences while keeping in mind that
the end "product" is a human that is counting on you to learn. Let me
know if you have any other questions.
